In [6]:
# upload sentiment_preprocessing.py if using colab
from sentiment_preprocessing import *

In [25]:
# Load datasets and parse dates

# convert date column from string to datetime
# convert stock time back to just a date to simplify closing price query
tweet_df1 = download_dataset_to_df("equinxx/stock-tweets-for-sentiment-analysis-and-prediction", "stock_tweets.csv")
tweet_df1['Date'] = pd.to_datetime(tweet_df1['Date'], utc=True)

stocks_df = download_dataset_to_df("equinxx/stock-tweets-for-sentiment-analysis-and-prediction", "stock_yfinance_data.csv")
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], utc=True).dt.date

tweet_df2 = download_dataset_to_df("ryanchan911/selective-stock-headlines-sentiment", "Project6500.csv")
tweet_df2['datetime'] = pd.to_datetime(tweet_df2['datetime'], utc=True, format='mixed', dayfirst=False).dt.date

In [26]:
# add column for the next day return using yfinance data
tweet_df1['return'] = tweet_df1.apply(lambda row: get_next_day_return(row['Date'], row['Stock Name'], stocks_df), axis=1)

# clean the tweets
tweet_df1['Tweet'] = tweet_df1['Tweet'].apply(clean_text)
tweet_df1

,Date,Tweet,Stock Name,Company Name,return
0,2022-09-29 23:41:16+00:00,mainstream media done amazing job brainwashing...,TSLA,"Tesla, Inc.",NaN
1,2022-09-29 23:24:43+00:00,tesla delivery estimates around 364k analysts ...,TSLA,"Tesla, Inc.",NaN
2,2022-09-29 23:18:08+00:00,3 even include 630m unvested rsus 630 addition...,TSLA,"Tesla, Inc.",NaN
3,2022-09-29 22:40:07+00:00,hahaha still trying stop tesla fsd bro get shi...,TSLA,"Tesla, Inc.",NaN
4,2022-09-29 22:27:05+00:00,stop trying kill kids sad deranged old man,TSLA,"Tesla, Inc.",NaN
...,...,...,...,...,...
80788,2021-10-07 17:11:57+00:00,fastest growing tech stocks market mix fintech...,XPEV,XPeng Inc.,-0.015682
80789,2021-10-04 17:05:59+00:00,earnings horizon quick snapshot largest increa...,XPEV,XPeng Inc.,0.041337
80790,2021-10-01 04:43:41+00:00,record delivery results testimony unwavering p...,XPEV,XPeng Inc.,NaN
80791,2021-10-01 00:03:32+00:00,delivered 10412 smart evs sep 2021 reaching mi...,XPEV,XPeng Inc.,NaN


In [10]:
# train val split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    list(tweet_df1['headline']), list(tweet_df1['label']), test_size=0.2, random_state=1
)

# tokenize data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(val_texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
print(train_encodings)
print("_______________________________________________________________________")
print(val_encodings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  9383,  2015,  ...,     0,     0,     0],
        [  101,  2662,  3748,  ...,     0,     0,     0],
        [  101,  9383,  2015,  ...,     0,     0,     0],
        ...,
        [  101,  9779, 24759,  ...,     0,     0,     0],
        [  101, 19387, 16545,  ...,     0,     0,     0],
        [  101,  6273,  2509,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
_______________________________________________________________________
{'input_ids': tensor([[  101, 26226,  2595,  ...,     0,     0,     0],
     